<h1 style="color: #492c68;">01 | BASICS</h1>

<h2 style="color: #327a81;">Libraries</h2>

In [1]:
## Basic libraries

import pandas as pd
import ast #
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

## Settings

pd.set_option('display.max_columns', None) # display all columns
import warnings
warnings.filterwarnings('ignore') # ignore warnings

<h2 style="color: #327a81;">Data Read</h2>

In [2]:
data= pd.read_csv("netflix_mood_recommender.csv")

In [3]:
df = data.copy()

<h1 style="color: #492c68;">MOOD RECOMMENDER</h1>

- Let's finally build our mood recommender. We will code a way to ask the user for a kind of mood and we'll show suggestions from our titles

In [4]:
# First let's check a sample our final df

df.sample(2)

,show_id,type,title,director,cast,country,date_added,rating,listed_in,n_seasons,movie_lenght,description,mood
3853,s3854,TV Show,I Have a Script,Not Provided,"Souad Abdullah, Shujoon, Fatima Al Safi",Not Provided,2019-05-07,TV-14,"International TV Shows, TV Comedies, TV Dramas",1.0,NaN,"After the death of a loved one, a woman decide...","sadness, disgust"
6376,s6377,Movie,Brick,Rian Johnson,"Joseph Gordon-Levitt, Nora Zehetner, Lukas Haa...",United States,2018-09-01,R,"Independent Movies, Thrillers",NaN,110.0,Teenage loner Brendan Fry navigates his high s...,"sadness, disgust"


In [5]:
df.shape

(8807, 13)

<h2 style="color: #327a81;">Mood one hot encoding</h2>

- Let's “hot code” the moods in the dataset so that our recommender works more efficiently.

In [6]:
# First we stack moods. One by one for each title

mood_encoding = df.drop(columns="mood").join(df["mood"].str.split(", ", expand=True).stack().reset_index(level=1, drop=True).rename("mood"))

In [7]:
# Then we apply "get dummies" to encode all mood possibilities

mood_encoding = pd.get_dummies(mood_encoding, columns=["mood"])*1

In [8]:
# Check out the encoding results

mood_encoding.head(2)

,show_id,type,title,director,cast,country,date_added,rating,listed_in,n_seasons,movie_lenght,description,mood_anger,mood_disgust,mood_fear,mood_joy,mood_sadness,mood_surprise
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Not Provided,United States,2021-09-25,PG-13,Documentaries,NaN,90.0,"As her father nears the end of his life, filmm...",0,0,0,0,1,0
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Not Provided,United States,2021-09-25,PG-13,Documentaries,NaN,90.0,"As her father nears the end of his life, filmm...",0,0,0,1,0,0


In [9]:
# Filtering the obtained dataframe to show all encoded moods by show id

mood_columns = ["mood_anger", "mood_disgust", "mood_fear", "mood_joy", "mood_sadness", "mood_surprise"]
total_moods = mood_encoding.groupby("show_id")[mood_columns].sum()

# And merge the og dataframe with the encoded moods by show id

df_moods = df.merge(total_moods, on="show_id", how="inner")
df_moods

,show_id,type,title,director,cast,country,date_added,rating,listed_in,n_seasons,movie_lenght,description,mood,mood_anger,mood_disgust,mood_fear,mood_joy,mood_sadness,mood_surprise
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Not Provided,United States,2021-09-25,PG-13,Documentaries,NaN,90.0,"As her father nears the end of his life, filmm...","sadness, joy",0,0,0,1,1,0
1,s2,TV Show,Blood & Water,Not Provided,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,TV-MA,"International TV Shows, TV Dramas, TV Mysteries",2.0,NaN,"After crossing paths at a party, a Cape Town t...","disgust, fear",0,1,1,0,0,0
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",Not Provided,2021-09-24,TV-MA,"Crime TV Shows, International TV Shows, TV Act...",1.0,NaN,To protect his family from a powerful drug lor...,"anger, disgust",1,1,0,0,0,0
3,s4,TV Show,Jailbirds New Orleans,Not Provided,Not Provided,Not Provided,2021-09-24,TV-MA,"Docuseries, Reality TV",1.0,NaN,"Feuds, flirtations and toilet talk go down amo...","disgust, joy",0,1,0,1,0,0
4,s5,TV Show,Kota Factory,Not Provided,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021-09-24,TV-MA,"International TV Shows, Romantic TV Shows, TV ...",2.0,NaN,In a city of coaching centers known to train I...,"disgust, fear",0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,2019-11-20,R,"Cult Movies, Dramas, Thrillers",NaN,158.0,"A political cartoonist, a crime reporter and a...","disgust, fear",0,1,1,0,0,0
8803,s8804,TV Show,Zombie Dumb,Not Provided,Not Provided,Not Provided,2019-07-01,TV-Y7,"Kids' TV, Korean TV Shows, TV Comedies",2.0,NaN,"While living alone in a spooky town, a young g...","joy, fear",0,0,1,1,0,0
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,2019-11-01,R,"Comedies, Horror Movies",NaN,88.0,Looking to survive in a world taken over by zo...,"fear, disgust",0,1,1,0,0,0
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,2020-01-11,PG,"Children & Family Movies, Comedies",NaN,88.0,"Dragged from civilian life, a former superhero...","disgust, fear",0,1,1,0,0,0


In [10]:
df_moods

,show_id,type,title,director,cast,country,date_added,rating,listed_in,n_seasons,movie_lenght,description,mood,mood_anger,mood_disgust,mood_fear,mood_joy,mood_sadness,mood_surprise
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,Not Provided,United States,2021-09-25,PG-13,Documentaries,NaN,90.0,"As her father nears the end of his life, filmm...","sadness, joy",0,0,0,1,1,0
1,s2,TV Show,Blood & Water,Not Provided,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,TV-MA,"International TV Shows, TV Dramas, TV Mysteries",2.0,NaN,"After crossing paths at a party, a Cape Town t...","disgust, fear",0,1,1,0,0,0
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",Not Provided,2021-09-24,TV-MA,"Crime TV Shows, International TV Shows, TV Act...",1.0,NaN,To protect his family from a powerful drug lor...,"anger, disgust",1,1,0,0,0,0
3,s4,TV Show,Jailbirds New Orleans,Not Provided,Not Provided,Not Provided,2021-09-24,TV-MA,"Docuseries, Reality TV",1.0,NaN,"Feuds, flirtations and toilet talk go down amo...","disgust, joy",0,1,0,1,0,0
4,s5,TV Show,Kota Factory,Not Provided,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021-09-24,TV-MA,"International TV Shows, Romantic TV Shows, TV ...",2.0,NaN,In a city of coaching centers known to train I...,"disgust, fear",0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,2019-11-20,R,"Cult Movies, Dramas, Thrillers",NaN,158.0,"A political cartoonist, a crime reporter and a...","disgust, fear",0,1,1,0,0,0
8803,s8804,TV Show,Zombie Dumb,Not Provided,Not Provided,Not Provided,2019-07-01,TV-Y7,"Kids' TV, Korean TV Shows, TV Comedies",2.0,NaN,"While living alone in a spooky town, a young g...","joy, fear",0,0,1,1,0,0
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,2019-11-01,R,"Comedies, Horror Movies",NaN,88.0,Looking to survive in a world taken over by zo...,"fear, disgust",0,1,1,0,0,0
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,2020-01-11,PG,"Children & Family Movies, Comedies",NaN,88.0,"Dragged from civilian life, a former superhero...","disgust, fear",0,1,1,0,0,0


<h2 style="color: #327a81;">Coding the recommender</h2>

In [26]:
def type_selection():
    while True:
        choice = input("Do you want to watch a Movie or a Tv Show?: ").strip().lower()
        if choice == "movie":
            type_mood = df_moods[df_moods["type"] == "Movie"]
            break
        elif choice == "tv show":
            type_mood =  df_moods[df_moods["type"] == "TV Show"]
            break
        else:
            print(f"{choice} is an invalid choice. Please choose either Movie or TV Show.")
            
    return type_mood

In [27]:
def mood_selection():
    
    moods = ["anger", "disgust", "fear", "joy", "sadness", "surprise"]
    
    while True:
        mood1 = input(f"Select a mood according to what you want to see ({', '.join(moods)}): ").strip().lower()
        if mood1 not in moods:
            print(f"{mood1} is not a valid choice. Please choose from ({', '.join(moods)}): ")
        else:
            break
    while True:
        mood2 = input(f"Select a second mood (this is optional, press enter to skip): ").strip().lower()
        if mood2 and mood2 not in moods:
            print(f"{mood2} is not a valid choice. Please choose from ({', '.join(moods)}) or press enter to skip: ")
        else:
            break
    
    return mood1, mood2

In [28]:
def mood_recommendation():
    
    mood1_column = f"mood_{mood1}"
    if mood2:
        mood2_column = f"mood_{mood2}"
        recommended = type_mood[(type_mood[mood1_column] == 1) & (type_mood[mood2_column] == 1)]
    else:
        recommended = type_mood[type_mood[mood1_column] == 1]
    return recommended

In [29]:
type_mood = type_selection()
mood1, mood2 = mood_selection()
recommended = mood_recommendation()
recommended

Do you want to watch a Movie or a Tv Show?:  movie
Select a mood according to what you want to see (anger, disgust, fear, joy, sadness, surprise):  anger
Select a second mood (this is optional, press enter to skip):  joy


,show_id,type,title,director,cast,country,date_added,rating,listed_in,n_seasons,movie_lenght,description,mood,mood_anger,mood_disgust,mood_fear,mood_joy,mood_sadness,mood_surprise
6,s7,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",Not Provided,2021-09-24,PG,Children & Family Movies,NaN,91.0,Equestria's divided. But a bright-eyed hero be...,"joy, anger",1,0,0,1,0,0
9,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,2021-09-24,PG-13,"Comedies, Dramas",NaN,104.0,A woman adjusting to life after a loss contend...,"joy, anger",1,0,0,1,0,0
119,s120,Movie,Here and There,JP Habac,"Janine Gutierrez, JC Santos, Victor Anastacio,...",Not Provided,2021-09-02,TV-MA,"Dramas, International Movies, Romantic Movies",NaN,99.0,After meeting through a heated exchange on soc...,"joy, anger",1,0,0,1,0,0
407,s408,Movie,Chhota Bheem & Ganesh,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jig...",Not Provided,2021-07-22,TV-Y7,Children & Family Movies,NaN,68.0,Bheem and Lord Ganesha join forces to save Dho...,"joy, anger",1,0,0,1,0,0
411,s412,Movie,Chhota Bheem And The Crown of Valhalla,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jig...",Not Provided,2021-07-22,TV-Y7,Children & Family Movies,NaN,64.0,To save the kingdom of Valhalla from invasion ...,"joy, anger",1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8181,s8182,Movie,The Adventure Club,Geoff Anderson,"Sam Ashe Arnold, Jakob Davies, Dalila Bela, Ro...",Canada,2017-05-10,TV-Y7,Children & Family Movies,NaN,88.0,"After finding a peculiar key, three smart and ...","joy, anger",1,0,0,1,0,0
8576,s8577,Movie,This Is Not What I Expected,Derek Hui,"Takeshi Kaneshiro, Zhou Dongyu, Sun Yizhou, Mi...",China,2017-10-14,TV-PG,"Comedies, International Movies, Romantic Movies",NaN,107.0,Romance and cuisine mix when a perpetually dis...,"joy, anger",1,0,0,1,0,0
8761,s8762,Movie,Wrong No.,Yasir Nawaz,"Javed Sheikh, Danish Taimoor, Janita Asma, Nay...",Pakistan,2017-07-01,TV-14,"Comedies, International Movies, Romantic Movies",NaN,133.0,Two identical strangers pursue their respectiv...,"joy, anger",1,0,0,1,0,0
8770,s8771,Movie,Yaadein,Subhash Ghai,"Jackie Shroff, Hrithik Roshan, Kareena Kapoor,...",India,2018-03-01,TV-14,"Dramas, International Movies, Romantic Movies",NaN,171.0,Two young lovers set out to overcome the obsta...,"anger, joy",1,0,0,1,0,0
